%pip install transformers==4.38.2
%pip install peft==0.10.0
%pip install sentencepiece==0.2.0
%pip install accelerate==0.28.0
%pip install -i https://pypi.org/simple/ bitsandbytes

%pip install --force-reinstall chromadb==0.4.23 
%pip install --force-reinstall llama_index==0.10.12
%pip install --force-reinstall sentence_transformers==2.2.2

In [1]:
from typing import List
import json
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, GenerationConfig
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
set_seed(42)

2024-07-04 23:11:17.703618: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 23:11:17.732288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 23:11:17.732315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 23:11:17.733246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 23:11:17.738230: I tensorflow/core/platform/cpu_feature_guar

In [2]:
MODEL_NAME = 'IlyaGusev/saiga_mistral_7b'

#MODEL_NAME ="openchat/openchat-3.5-0106"

EMBED_MODEL = 'intfloat/e5-large-v2'
#EMBED_MODEL = 'BAAI/bge-small-en-v1.5'

In [3]:
import torch

# Get the properties of the CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
properties = torch.cuda.get_device_properties(device)

# Print the total memory and memory available on the CUDA device
total_memory = properties.total_memory / 1024**2  # Convert to megabytes
memory_available = torch.cuda.memory_reserved(device) / 1024**2  # Convert to megabytes

print(f"Total video memory: {total_memory:.2f} MB")
print(f"Available video memory: {memory_available:.2f} MB")

Total video memory: 7951.19 MB
Available video memory: 0.00 MB


Иницилизируем модель для получения эмббедингов

In [4]:
embed_model = SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL, 
                                                device='cuda'
                                                )

Создаем коллекцию в векторном хранилище

In [5]:
client = chromadb.Client()
collection = client.get_or_create_collection("RAG", embedding_function=embed_model,
    metadata={"hnsw:space": "cosine"} 
    )
client.list_collections()

[Collection(name=RAG)]

Загружаем данные

In [6]:
#file_path = "triples_ft_pp.jsonl"
file_path = "data/movie_ground_truth.jsonl" # больше данных - больше вариативность

with open(file_path, "r") as f:
    data = f.readlines()

In [7]:
documents = []
for i_text in data:
    i_text = json.loads(i_text)
    documents.append(i_text['sent'])

In [8]:
metadata = [{'subject': 'movie'} for i in range(len(documents))]

### Грузим каталог

In [9]:
df = pd.read_csv('../lesson12/data/data.csv_')
df.tail(20)

,title,url,price,text
25,ЭМ компостер для пищевых отходов,https://em-russia.ru/shop/all/%D0%AD%D0%9C-%D0...,2340,ЭМ-контейнер - биоутилизатор органических отхо...
26,Тоник для кожи лица,https://em-russia.ru/shop/all/tonik-dlya-kozhi...,3025,Разнообразные натуральные свойства и процесс ф...
27,Зубная паста,https://em-russia.ru/shop/all/zubnaya-pasta,26250,Лечебно-профилактическая зубная паста с мягким...
28,Защитная ЭМ маска,https://em-russia.ru/shop/all/zashchitnaya-em-...,195,Индивидуальная многоразовая защитная маска обр...
29,Набор для ЭМ компоста,https://em-russia.ru/shop/all/nabor-dlya-em-ko...,1600,Набор для приготовления ЭМ-компоста\nСостав\nС...
30,ЭМ мыло жидкое органическое ПРЕМИУМ,https://em-russia.ru/shop/all/em-mylo-zhidkoe-...,200,"Мыло жидкое для рук и тела ""ЭМ-МЫЛО Премиум"" и..."
31,Средство для уборки ЭМ Спрей,https://em-russia.ru/shop/all/sredstvo-dlya-ub...,5380,Устраняет неприятные запахи (в том числе запах...
32,Средство дезинфицирующее ЭМ Вита Про,https://em-russia.ru/shop/all/sredstvo-dezinfi...,5380,Препарат «ЭМ-Вита.Про» предназначен для устран...
33,Биодезинфектант БиоЭМ АКТИВ,https://em-russia.ru/shop/all/biodezinfektant-...,1050,"Препарат полностью природного происхождения, в..."
34,Средство для уборки ЭМ Спрей,https://em-russia.ru/shop/all/sredstvo-dlya-ub...,425,Устраняет неприятные запахи (в том числе запах...


In [10]:
documents = []
metadata = []
for index, row in df.iterrows():
    title = row['title']
    text = row['text']
    documents.append(f'{title}\n{text}')
    metadata.append({'url' : row['url']})

### Грузим статьи

In [11]:
import os
for filename in os.listdir('../lesson12/data/page_texts/'):
    with open('../lesson12/data/page_texts/' + filename, 'r') as f:
        sentenses = f.read()
        
        #for sentense in sentenses:
        documents.append(sentenses)
        metadata.append({'url' : f'https://em-russia.ru/base/{filename.split(".")[0]}/'})
        
    

Подготоавливаем данные для загрузки в векторную бд

In [12]:
#metadata = [{'type': 'product'} for i in range(len(documents))]

In [13]:
ids = ['id'+str(collection.count()+i+1) for i in range(len(documents))]

In [14]:
collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids,
)


In [15]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
torch_device = "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, 
                                             pad_token_id=tokenizer.eos_token_id).to(torch_device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# model.generation_config = GenerationConfig(
#        do_sample=True,
#        top_k=50,
#        top_p=0.95,
#        num_return_sequences=num_return_sequences, 
#        num_beams=num_beams,
#        temperature=0.5
#    )

Промпт для RAG

In [17]:
prompt = """Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query}
Answer:"""

Функция для поиска контекста

In [27]:
def get_context(query):
    context_raw = collection.query(query_texts=query, n_results=3)
    context = "\n".join(context_raw['documents'][0])
    context = context + " Подробнее по ссылке " + context_raw['metadatas'][0][0]['url']
    return context

In [19]:
query = "Чем лучше убирать помещения?"
context_raw = collection.query(query_texts=query, n_results=3)
context_raw

{'ids': [['id123', 'id298', 'id89']],
 'distances': [[0.18283700942993164,
   0.18444430828094482,
   0.18444430828094482]],
 'metadatas': [[{'url': 'https://em-russia.ru/base/eko-uborka-s-em-nedelya-karantina-s-polzoy-i-profilaktikoy/'},
   {'url': 'https://em-russia.ru/base/kompleksnoe-primenenie-em-v-restorane-peru/'},
   {'url': 'https://em-russia.ru/base/kompleksnoe-primenenie-em-v-restorane-peru2/'}]],
 'embeddings': None,
 'documents': [['ЭКО-уборка с ЭМ: неделя карантина с пользой и профилактикой 30.03.2020Впереди целая Неделя дома.Чем займётесь?Ну наверняка же будете убираться. Это прям повод генеральную уборку провести.Можно сразу убить двух зайцев - убраться качественно и экологично без химии, и провести биодезинфекцию воздуха и пространства. В общем - делаем одно дело, а вопросом решаем сразу несколько.Препарат ЭМ-Спрей Вам в помощь!ИНСТРУКЦИЯ к ЭКО-уборке:1. Разбавляем #эмспрей в концентрации 1:500 и аэрозольно из мелкодисперсного распылителя распыляем на люстры, шторы, ст

In [28]:
get_context(query)

'ЭКО-уборка с ЭМ: неделя карантина с пользой и профилактикой 30.03.2020Впереди целая Неделя дома.Чем займётесь?Ну наверняка же будете убираться. Это прям повод генеральную уборку провести.Можно сразу убить двух зайцев - убраться качественно и экологично без химии, и провести биодезинфекцию воздуха и пространства. В общем - делаем одно дело, а вопросом решаем сразу несколько.Препарат ЭМ-Спрей Вам в помощь!ИНСТРУКЦИЯ к ЭКО-уборке:1. Разбавляем #эмспрей в концентрации 1:500 и аэрозольно из мелкодисперсного распылителя распыляем на люстры, шторы, стены, ковры, мягкую мебель, гардеробная, обувь и т.д. от пыли и для БИО дезинфекции воздуха и пространства.2. Остался разбавленный раствор? Добавляем в ведро с водой для мытья пола, чтобы он был чистым и меньше притягивал к себе частицы пыли, а также можем добавить в сливы раковины и душевой кабины, чтобы сделать био-прочистку или профилактику от засоров и для устранения запахов.3. Делаем концентрацию 1:100 и моем кафель, сантехнику, панели на ку

Функция для генерации нескольких ответов модели

Закомментированные параметры - эксперимент

In [29]:
def get_model_output(context, query):
    model_inputs = tokenizer(prompt.format(context=context, query=query), return_tensors='pt').to(torch_device)

    sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=50,
    do_sample=False, # True,
    #top_k=50,
    #top_p=0.95,
    num_return_sequences=3, #6, 
    num_beams=3, #6,
    temperature=None
    )

    result = []
    for i, output in enumerate(sample_outputs):
        result.append(tokenizer.decode(output, skip_special_tokens=True).split("Answer:")[1].strip().split("\n")[0].strip())
    return result

In [30]:
def get_output_texts(query: str) -> List[str]:
    context = get_context(query)
    output_texts = get_model_output(context, query)
    return output_texts

Тест

In [31]:
get_output_texts("Чем лучше убирать помещения?")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


['Лучше всего убирать помещения с использованием эффективных микроорганизмов (ЭМ). ЭМ способны утилизировать пыль, загряз',
 'Лучше всего убирать помещения с использованием эффективных микроорганизмов (ЭМ). ЭМ могут убирать пыль, грязь, пятна',
 'Лучше всего убирать помещения с использованием эффективных микроорганизмов (ЭМ). ЭМ способны утилизировать органические загрязнения,']